# Analyzing fire payrolls in L.A.: 2011-2018

This notebook processess government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [ ]:
import altair as alt
import altair_latimes as lat
import pandas as pd
import geopandas as gpd
import cpi
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format
cpi.update()

### Import fire payroll data

In [ ]:
# processed in 01-california-payroll-descriptives
agency_ot = pd.read_csv('output/agency_overtime.csv')
agency_ot_share = pd.read_csv('output/agency_overtime_share_ot.csv')
payroll_fire = pd.read_csv('/Users/mhustiles/data/data/controller/output/payroll_fire.csv')

---

### Filter dataframe to include only Los Angeles city and county fire

In [ ]:
la = pd.DataFrame(payroll_fire[payroll_fire['employer'] == 'LOS ANGELES'])

In [ ]:
la.employerfull = la.employerfull.str.replace('_', ' ')

In [ ]:
la_agencies = la.groupby(['employerfull', 'year', 'type'])\
    .agg({'adjusted_overtime':'sum'}).reset_index()

In [ ]:
la_agencies

In [ ]:
alt.Chart(la_agencies).mark_line().encode(
    x='year:O',
    y='adjusted_overtime',
    color='employerfull'
)

In [ ]:
la_agencies.to_csv('output/la_agencies.csv')

---

### Create a dataframe combining and pivoting listing all the large places

In [ ]:
la_pivot_ot = pd.DataFrame(pd.pivot_table(la_agencies, \
                values='adjusted_overtime', index=['employerfull', 'type'], columns=['year']).reset_index().fillna(0))

In [ ]:
la_pivot_ot.columns = la_pivot_ot.columns.map(str)

### What did each fire department pay in overtime (just to firefighter titles) each year? 

In [ ]:
la_pivot_ot['ot_pct_change'] = \
                ((la_pivot_ot['2018']-la_pivot_ot['2011'])/\
                 la_pivot_ot['2011'])*100

In [ ]:
la_pivot_ot.head()

In [ ]:
#2011
'${:,.0f}'.format((la_pivot_ot.iloc[1,2] + la_pivot_ot.iloc[0,2]))

In [ ]:
#2018
'${:,.0f}'.format((la_pivot_ot.iloc[1,9] + la_pivot_ot.iloc[0,9]))

In [ ]:
'{:,.2f}%'.format(((la_pivot_ot.iloc[1,9] + la_pivot_ot.iloc[0,9])-(la_pivot_ot.iloc[1,2] + la_pivot_ot.iloc[0,2]))\
        / (la_pivot_ot.iloc[1,2] + la_pivot_ot.iloc[0,2])*100)

In [ ]:
la_pivot_ot.to_csv('output/la_city_county.csv')

---

### Charting firefighter wages and benefits in Los Angeles

In [ ]:
la_all = la.groupby(['year', 'employerfull']).agg({'adjusted_overtime':'sum', \
                                            'adjusted_basewages':'sum', \
                                        'adjusted_wages':'sum', \
                                            'adjusted_benefits':'sum'\
                                            }).reset_index()

### Clean up the column names

In [ ]:
la_all.rename(columns={'adjusted_overtime': 'Overtime',\
                    'adjusted_basewages': 'BaseWages',\
                   'adjusted_wages': 'Wages',\
                   'adjusted_benefits':'Benefits',\
                     'year': 'Year',
                      'employerfull': 'Place'}, inplace=True)

### Melt the dataframe for charting

In [ ]:
la_melt = pd.melt(la_all,id_vars=['Year', 'Place'], value_vars=['Overtime','BaseWages'],\
        var_name='Compensation', value_name='Total')

### Chart the overtime amount in annual dollars

In [ ]:
otdollars = alt.Chart(la_melt).mark_area().encode(
    x=alt.X("Year:N", title=' '),
    y=alt.Y("Total:Q", title=' ', axis=alt.Axis(tickCount=6, format='$,s')),
    color=alt.Color("Compensation:N", title=''),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'Compensation',
      sort='ascending'
    ),
    facet=alt.Facet('Place:N', title='Overtime in inflation-adjusted dollars')
).properties(width=300, height=300, columns=2, title='Fireighter pay')

otdollars.configure_header(
    titleColor='gray',
    titleFontSize=14,
    titleFontWeight = 'normal',
    titleOrient = 'top',
    titleAnchor='middle',
    titlePadding=10,
    labelFontSize=11
).configure_legend(
    orient='right'
)

In [ ]:
la_melt.sort_values(by=['Place', 'Compensation', 'Year'], ascending=True)

### Normalized version

In [ ]:
otshare = alt.Chart(la_melt).mark_area().encode(
    x=alt.X("Year:N", title=' '),
    y=alt.Y("Total:Q", stack="normalize", title=' ', axis=alt.Axis(tickCount=6, format='%')),
    color="Compensation:N",
    order=alt.Order(
      # Sort the segments of the bars by this field
      'Compensation',
      sort='ascending'
    ),
    facet=alt.Facet('Place:N', title=' ')
).properties(width=300, height=300, columns=2, title='Firefighter compensation in Los Angeles')

otshare.configure_header(
    titleColor='gray',
    titleFontSize=14,
    titleFontWeight = 'normal',
    titleOrient = 'top',
    titleAnchor='middle',
    titlePadding=10,
    labelFontSize=11
).configure_legend(
    orient='right'
)

In [ ]:
la_melt.sort_values(by=['Place', 'Compensation']).to_csv('output/la_melt.csv')

---

### How many firefighters received six-figure amounts of overtime in a year?

In [ ]:
la_highest = la[la['overtime'] > 200000]

In [ ]:
la_high = la[la['overtime'] > 100000]

In [ ]:
print('$200k:',len(la_highest),'| $100k:',len(la_high)) 

### Group by agency type and year and count recipients of $100k overtime

In [ ]:
la_high_years = la_high.groupby(['year','type']).agg('size').reset_index(name='count')
la_high_years.sort_values(by='type', ascending=False).head(25)

### Group by agency type and year and count recipients of $200k overtime

In [ ]:
la_highest_years = la_highest.groupby(['year','type']).agg('size').reset_index(name='count')
la_highest_years.sort_values(by='type', ascending=False).head(25)

### Group by agency type and year and count recipients of $200k overtime 

In [ ]:
la_highest_years = la_highest.groupby(['year','type']).agg('size').reset_index(name='count')

In [ ]:
alt.Chart(la_highest_years).mark_bar().encode(
    x=alt.X("year:N", title=' '),
    y=alt.Y("mean(count):Q", title=' ', axis=alt.Axis(tickCount=6, format='')),
    facet='type:O'
).properties(width=300, height=300, columns=2, title='L.A. firefighters > $200k OT')

In [ ]:
la_highest_years.sort_values(by=['type','year'], ascending=False).head(20)

In [ ]:
la_high_positions = la_high.groupby('position').agg({'adjusted_overtime':'sum', \
    'adjusted_basewages':'sum', \
    'adjusted_wages':'sum', \
    'adjusted_benefits':'sum'\
}).reset_index()

### Payroll by title

In [ ]:
la_high_positions.sort_values(by='adjusted_overtime', ascending=False)

---

### $300,000 salaries

In [ ]:
threehundred = payroll_fire[(payroll_fire.adjusted_wages > 300000) &\
                            (payroll_fire.year == 2011) &\
                            (payroll_fire.employer.str.contains('LOS ANGELES'))]

In [ ]:
threehundred.head()

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx